In [2]:
import pandas as pd
import numpy as np
from pathlib import Path
import warnings
import joblib
import time
warnings.filterwarnings('ignore')

# ML Libraries
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import (
    RandomForestClassifier, 
    ExtraTreesClassifier, 
    GradientBoostingClassifier,
    HistGradientBoostingClassifier,
    AdaBoostClassifier
)
import xgboost as xgb
import lightgbm as lgb
from catboost import CatBoostClassifier

# Paths
BASE_DIR = Path.cwd().parent if (Path.cwd() / 'ml_models_train').exists() == False else Path.cwd()
if (BASE_DIR / 'data').exists() == False:
    BASE_DIR = Path.cwd().parent
DATA_DIR = BASE_DIR / 'data'
MODEL_DIR = BASE_DIR / 'models' / 'signal_generator_v11'
MODEL_DIR.mkdir(parents=True, exist_ok=True)

# GPU Check
try:
    import torch
    GPU_AVAILABLE = torch.cuda.is_available()
except:
    GPU_AVAILABLE = False

print("="*70)
print("🚀 FOREX SIGNAL GENERATOR V11")
print("   Multi-Class: BUY / SELL / HOLD")
print("   10+ Model Ensemble")
print("="*70)
print(f"✓ GPU Available: {GPU_AVAILABLE}")
print(f"✓ Data Directory: {DATA_DIR}")
print(f"✓ Model Directory: {MODEL_DIR}")

🚀 FOREX SIGNAL GENERATOR V11
   Multi-Class: BUY / SELL / HOLD
   10+ Model Ensemble
✓ GPU Available: True
✓ Data Directory: c:\Users\Acer\Desktop\Forex-Signal-App\data
✓ Model Directory: c:\Users\Acer\Desktop\Forex-Signal-App\models\signal_generator_v11


## 1. Data Loading

In [3]:
# Load Data
train_df = pd.read_csv(DATA_DIR / 'EUR_USD_1min.csv')
test_df = pd.read_csv(DATA_DIR / 'EUR_USD_test.csv')

for df in [train_df, test_df]:
    if 'timestamp' in df.columns:
        df.rename(columns={'timestamp': 'time'}, inplace=True)
    df['time'] = pd.to_datetime(df['time'])

print(f"Train: {len(train_df):,} rows")
print(f"Test: {len(test_df):,} rows")
print(f"Train period: {train_df['time'].min()} to {train_df['time'].max()}")
print(f"Test period: {test_df['time'].min()} to {test_df['time'].max()}")

Train: 1,859,492 rows
Test: 296,778 rows
Train period: 2019-12-31 16:00:00+00:00 to 2024-12-30 16:00:00+00:00
Test period: 2024-12-31 16:00:00+00:00 to 2025-10-17 06:11:00+00:00


## 2. V11 Feature Engineering

In [4]:
def add_features_v11(df):
    """
    V11 Features: V10 base + Enhanced for Multi-Class
    """
    df = df.copy()
    
    # ==================== TIME FEATURES ====================
    df['hour'] = df['time'].dt.hour
    df['day_of_week'] = df['time'].dt.dayofweek
    df['is_london'] = ((df['hour'] >= 8) & (df['hour'] < 16)).astype(int)
    df['is_ny'] = ((df['hour'] >= 13) & (df['hour'] < 21)).astype(int)
    df['is_overlap'] = ((df['hour'] >= 13) & (df['hour'] < 16)).astype(int)
    df['session_quality'] = df['is_london'] + df['is_ny'] + df['is_overlap'] * 2
    
    # ==================== MOVING AVERAGES ====================
    for p in [5, 10, 20, 50, 100, 200]:
        df[f'sma_{p}'] = df['close'].rolling(p).mean()
        df[f'ema_{p}'] = df['close'].ewm(span=p, adjust=False).mean()
    
    # MA Crosses
    df['ema_5_10_cross'] = (df['ema_5'] - df['ema_10']) / df['ema_10'] * 100
    df['ema_10_20_cross'] = (df['ema_10'] - df['ema_20']) / df['ema_20'] * 100
    df['ema_20_50_cross'] = (df['ema_20'] - df['ema_50']) / df['ema_50'] * 100
    df['sma_50_200_cross'] = (df['sma_50'] - df['sma_200']) / df['sma_200'] * 100
    
    # ==================== RSI ====================
    delta = df['close'].diff()
    gain = delta.where(delta > 0, 0).rolling(14).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(14).mean()
    rs = gain / (loss + 1e-10)
    df['rsi'] = 100 - (100 / (1 + rs))
    df['rsi_oversold'] = (df['rsi'] < 30).astype(int)
    df['rsi_overbought'] = (df['rsi'] > 70).astype(int)
    
    # ==================== MACD ====================
    ema12 = df['close'].ewm(span=12).mean()
    ema26 = df['close'].ewm(span=26).mean()
    df['macd'] = ema12 - ema26
    df['macd_signal'] = df['macd'].ewm(span=9).mean()
    df['macd_hist'] = df['macd'] - df['macd_signal']
    df['macd_cross_up'] = ((df['macd'] > df['macd_signal']) & (df['macd'].shift(1) <= df['macd_signal'].shift(1))).astype(int)
    df['macd_cross_down'] = ((df['macd'] < df['macd_signal']) & (df['macd'].shift(1) >= df['macd_signal'].shift(1))).astype(int)
    
    # ==================== BOLLINGER BANDS ====================
    df['bb_mid'] = df['close'].rolling(20).mean()
    df['bb_std'] = df['close'].rolling(20).std()
    df['bb_upper'] = df['bb_mid'] + 2 * df['bb_std']
    df['bb_lower'] = df['bb_mid'] - 2 * df['bb_std']
    df['bb_width'] = (df['bb_upper'] - df['bb_lower']) / (df['bb_mid'] + 1e-10)
    df['bb_position'] = (df['close'] - df['bb_lower']) / (df['bb_upper'] - df['bb_lower'] + 1e-10)
    
    # ==================== ATR & ADX ====================
    df['tr0'] = abs(df['high'] - df['low'])
    df['tr1'] = abs(df['high'] - df['close'].shift())
    df['tr2'] = abs(df['low'] - df['close'].shift())
    df['tr'] = df[['tr0', 'tr1', 'tr2']].max(axis=1)
    
    df['up_move'] = df['high'] - df['high'].shift()
    df['down_move'] = df['low'].shift() - df['low']
    
    df['plus_dm'] = np.where((df['up_move'] > df['down_move']) & (df['up_move'] > 0), df['up_move'], 0)
    df['minus_dm'] = np.where((df['down_move'] > df['up_move']) & (df['down_move'] > 0), df['down_move'], 0)
    
    period = 14
    df['atr'] = df['tr'].rolling(period).mean()
    df['plus_di'] = 100 * (df['plus_dm'].rolling(period).mean() / (df['atr'] + 1e-10))
    df['minus_di'] = 100 * (df['minus_dm'].rolling(period).mean() / (df['atr'] + 1e-10))
    df['dx'] = 100 * abs(df['plus_di'] - df['minus_di']) / (df['plus_di'] + df['minus_di'] + 1e-10)
    df['adx'] = df['dx'].rolling(period).mean()
    df['di_diff'] = df['plus_di'] - df['minus_di']
    
    # ==================== CCI ====================
    tp = (df['high'] + df['low'] + df['close']) / 3
    sma_tp = tp.rolling(20).mean()
    mad_tp = tp.rolling(20).apply(lambda x: np.abs(x - x.mean()).mean())
    df['cci'] = (tp - sma_tp) / (0.015 * mad_tp + 1e-10)
    
    # ==================== WILLIAMS %R ====================
    hh = df['high'].rolling(14).max()
    ll = df['low'].rolling(14).min()
    df['williams_r'] = -100 * (hh - df['close']) / (hh - ll + 1e-10)
    
    # ==================== STOCHASTIC ====================
    df['stoch_k'] = 100 * (df['close'] - ll) / (hh - ll + 1e-10)
    df['stoch_d'] = df['stoch_k'].rolling(3).mean()
    
    # ==================== VOLATILITY ====================
    df['returns'] = df['close'].pct_change()
    df['volatility'] = df['returns'].rolling(20).std() * 100
    df['volatility_sma'] = df['volatility'].rolling(50).mean()
    df['volatility_ratio'] = df['volatility'] / (df['volatility_sma'] + 1e-10)
    
    # ==================== PRICE ACTION ====================
    df['body'] = df['close'] - df['open']
    df['upper_wick'] = df['high'] - df[['open', 'close']].max(axis=1)
    df['lower_wick'] = df[['open', 'close']].min(axis=1) - df['low']
    df['body_ratio'] = abs(df['body']) / (df['high'] - df['low'] + 1e-10)
    df['is_bullish'] = (df['close'] > df['open']).astype(int)
    df['bullish_streak'] = df['is_bullish'].rolling(5).sum()
    
    # ==================== SUPPORT/RESISTANCE ====================
    df['high_20'] = df['high'].rolling(20).max()
    df['low_20'] = df['low'].rolling(20).min()
    df['dist_to_high'] = (df['high_20'] - df['close']) / (df['atr'] + 1e-10)
    df['dist_to_low'] = (df['close'] - df['low_20']) / (df['atr'] + 1e-10)
    
    # ==================== MOMENTUM ====================
    for p in [5, 10, 20, 50]:
        df[f'momentum_{p}'] = (df['close'] - df['close'].shift(p)) / (df['atr'] + 1e-10)
    
    # ==================== COMPOSITE SCORES ====================
    # Trend Score (-4 to +4)
    df['trend_score'] = (
        (df['close'] > df['sma_20']).astype(int) - (df['close'] < df['sma_20']).astype(int) +
        (df['sma_20'] > df['sma_50']).astype(int) - (df['sma_20'] < df['sma_50']).astype(int) +
        (df['sma_50'] > df['sma_200']).astype(int) - (df['sma_50'] < df['sma_200']).astype(int) +
        (df['di_diff'] > 0).astype(int) - (df['di_diff'] < 0).astype(int)
    )
    
    # Momentum Score (-4 to +4)
    df['momentum_score'] = (
        (df['rsi'] > 50).astype(int) - (df['rsi'] < 50).astype(int) +
        (df['macd_hist'] > 0).astype(int) - (df['macd_hist'] < 0).astype(int) +
        (df['cci'] > 0).astype(int) - (df['cci'] < 0).astype(int) +
        (df['stoch_k'] > 50).astype(int) - (df['stoch_k'] < 50).astype(int)
    )
    
    # Volatility State
    df['volatility_state'] = np.where(
        df['volatility_ratio'] > 1.5, 2,
        np.where(df['volatility_ratio'] < 0.5, 0, 1)
    )
    
    # ==================== CLEANUP ====================
    drop_cols = ['tr0', 'tr1', 'tr2', 'tr', 'up_move', 'down_move', 'plus_dm', 'minus_dm', 'dx']
    df.drop(columns=[c for c in drop_cols if c in df.columns], inplace=True)
    
    return df

print("Adding V11 features...")
train_df = add_features_v11(train_df)
test_df = add_features_v11(test_df)
print(f"✓ Features added. Total columns: {len(train_df.columns)}")

Adding V11 features...
✓ Features added. Total columns: 72
✓ Features added. Total columns: 72


## 3. V11 Multi-Class Labeling (BUY / SELL / HOLD)

In [5]:
def create_labels_v11(df, forward_periods=60, min_pips=15, ratio=1.5):
    """
    V11 Multi-Class Labeling:
    - BUY (1): Strong upward movement
    - SELL (2): Strong downward movement  
    - HOLD (0): No clear direction
    """
    df = df.copy()
    min_move = min_pips * 0.0001
    
    df['future_max'] = df['high'].rolling(forward_periods).max().shift(-forward_periods)
    df['future_min'] = df['low'].rolling(forward_periods).min().shift(-forward_periods)
    
    df['up_move'] = df['future_max'] - df['close']
    df['down_move'] = df['close'] - df['future_min']
    
    # Multi-class conditions
    conditions = [
        (df['up_move'] >= min_move) & (df['up_move'] > df['down_move'] * ratio),    # BUY
        (df['down_move'] >= min_move) & (df['down_move'] > df['up_move'] * ratio),  # SELL
    ]
    choices = [1, 2]  # 1=BUY, 2=SELL, 0=HOLD
    df['signal'] = np.select(conditions, choices, default=0)
    
    df.drop(['future_max', 'future_min', 'up_move', 'down_move'], axis=1, inplace=True)
    return df

# Create labels
train_df = create_labels_v11(train_df, forward_periods=60, min_pips=15, ratio=1.5)
test_df = create_labels_v11(test_df, forward_periods=60, min_pips=15, ratio=1.5)

# Remove NaN rows
train_df = train_df.dropna().copy()
test_df = test_df.dropna().copy()

# Class distribution
print("\n📊 Class Distribution:")
print("\nTrain:")
train_counts = train_df['signal'].value_counts().sort_index()
for sig, count in train_counts.items():
    label = {0: 'HOLD', 1: 'BUY', 2: 'SELL'}[sig]
    print(f"  {label}: {count:,} ({count/len(train_df)*100:.1f}%)")

print("\nTest:")
test_counts = test_df['signal'].value_counts().sort_index()
for sig, count in test_counts.items():
    label = {0: 'HOLD', 1: 'BUY', 2: 'SELL'}[sig]
    print(f"  {label}: {count:,} ({count/len(test_df)*100:.1f}%)")


📊 Class Distribution:

Train:
  HOLD: 1,466,044 (78.8%)
  BUY: 193,796 (10.4%)
  SELL: 199,453 (10.7%)

Test:
  HOLD: 216,283 (72.9%)
  BUY: 41,895 (14.1%)
  SELL: 38,401 (12.9%)


## 4. Feature Selection

In [6]:
# Prepare Data
exclude_cols = ['time', 'signal', 'open', 'high', 'low', 'close', 'volume', 'tick_volume']
feature_cols = [c for c in train_df.columns if c not in exclude_cols]

X_train_full = train_df[feature_cols].values
y_train = train_df['signal'].values
X_test_full = test_df[feature_cols].values
y_test = test_df['signal'].values

print(f"Features before selection: {len(feature_cols)}")

# Feature Importance based selection using LightGBM
lgb_params = {'n_estimators': 200, 'max_depth': 6, 'random_state': 42, 'verbose': -1}
if GPU_AVAILABLE:
    lgb_params['device'] = 'gpu'

selector_model = lgb.LGBMClassifier(**lgb_params)
selector_model.fit(X_train_full, y_train)

# Get feature importances
importances = selector_model.feature_importances_
importance_df = pd.DataFrame({
    'feature': feature_cols,
    'importance': importances
}).sort_values('importance', ascending=False)

# Select top features using median threshold
threshold = np.median(importances)
selected_features = importance_df[importance_df['importance'] >= threshold]['feature'].tolist()

print(f"Features after selection: {len(selected_features)}")
print(f"\nTop 20 Features:")
print(importance_df.head(20).to_string(index=False))

Features before selection: 66
Features after selection: 33

Top 20 Features:
         feature  importance
            hour        2614
sma_50_200_cross        1681
  volatility_sma        1563
     day_of_week        1037
         sma_200         962
             atr         781
         ema_200         654
 ema_20_50_cross         574
         sma_100         534
volatility_ratio         473
         high_20         467
      volatility         452
     momentum_50         411
          low_20         376
        bb_lower         358
        bb_upper         356
             adx         317
         ema_100         311
          sma_50         285
     macd_signal         252
Features after selection: 33

Top 20 Features:
         feature  importance
            hour        2614
sma_50_200_cross        1681
  volatility_sma        1563
     day_of_week        1037
         sma_200         962
             atr         781
         ema_200         654
 ema_20_50_cross         574
      

## 5. Prepare Final Data

In [7]:
# Use selected features
X_train = train_df[selected_features].values
X_test = test_df[selected_features].values

# Scale
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(f"Training data: {X_train_scaled.shape}")
print(f"Test data: {X_test_scaled.shape}")
print(f"Classes: {np.unique(y_train)}")

Training data: (1859293, 33)
Test data: (296579, 33)
Classes: [0 1 2]


## 6. V11 Model Training (10+ Diverse Models)

In [8]:
print("="*70)
print("🚀 TRAINING V11 MODELS (10+ Diverse Ensemble)")
print("="*70)

models = {}

# ==================== XGBOOST MODELS ====================
xgb_base_params = {
    'objective': 'multi:softprob',
    'num_class': 3,
    'eval_metric': 'mlogloss',
    'verbosity': 0
}
if GPU_AVAILABLE:
    xgb_base_params.update({'tree_method': 'hist', 'device': 'cuda'})

# XGBoost 1 - Balanced
models['xgb1'] = xgb.XGBClassifier(
    n_estimators=300, max_depth=6, learning_rate=0.05,
    subsample=0.8, colsample_bytree=0.8,
    reg_alpha=0.1, reg_lambda=1.0,
    min_child_weight=3, random_state=42,
    **xgb_base_params
)

# XGBoost 2 - Deeper
models['xgb2'] = xgb.XGBClassifier(
    n_estimators=250, max_depth=8, learning_rate=0.06,
    subsample=0.75, colsample_bytree=0.75,
    reg_alpha=0.05, reg_lambda=0.5,
    gamma=0.1, random_state=43,
    **xgb_base_params
)

# XGBoost 3 - Conservative
models['xgb3'] = xgb.XGBClassifier(
    n_estimators=400, max_depth=4, learning_rate=0.04,
    subsample=0.85, colsample_bytree=0.85,
    reg_alpha=0.2, reg_lambda=2.0,
    min_child_weight=5, random_state=44,
    **xgb_base_params
)

# ==================== LIGHTGBM MODELS ====================
lgb_base_params = {'verbose': -1, 'objective': 'multiclass', 'num_class': 3}
if GPU_AVAILABLE:
    lgb_base_params['device'] = 'gpu'

# LightGBM 1 - Balanced
models['lgb1'] = lgb.LGBMClassifier(
    n_estimators=300, max_depth=6, learning_rate=0.05,
    subsample=0.8, colsample_bytree=0.8,
    num_leaves=31, min_child_samples=30,
    reg_alpha=0.1, reg_lambda=1.0,
    random_state=42, **lgb_base_params
)

# LightGBM 2 - More leaves
models['lgb2'] = lgb.LGBMClassifier(
    n_estimators=250, max_depth=8, learning_rate=0.06,
    subsample=0.75, colsample_bytree=0.75,
    num_leaves=63, min_child_samples=20,
    reg_alpha=0.05, reg_lambda=0.5,
    random_state=45, **lgb_base_params
)

# ==================== CATBOOST MODELS ====================
cat_base_params = {'loss_function': 'MultiClass', 'verbose': False}
if GPU_AVAILABLE:
    cat_base_params.update({'task_type': 'GPU', 'devices': '0'})

# CatBoost 1
models['cat1'] = CatBoostClassifier(
    iterations=300, depth=6, learning_rate=0.05,
    l2_leaf_reg=3.0, random_strength=0.5,
    bagging_temperature=0.5, random_seed=42,
    **cat_base_params
)

# CatBoost 2
models['cat2'] = CatBoostClassifier(
    iterations=250, depth=8, learning_rate=0.06,
    l2_leaf_reg=2.0, random_strength=0.3,
    bagging_temperature=0.3, random_seed=46,
    **cat_base_params
)

# ==================== SKLEARN ENSEMBLE MODELS ====================
# RandomForest
models['rf'] = RandomForestClassifier(
    n_estimators=200, max_depth=10, min_samples_split=10,
    min_samples_leaf=5, max_features='sqrt',
    random_state=42, n_jobs=-1
)

# ExtraTrees
models['et'] = ExtraTreesClassifier(
    n_estimators=200, max_depth=10, min_samples_split=10,
    min_samples_leaf=5, max_features='sqrt',
    random_state=43, n_jobs=-1
)

# HistGradientBoosting (Native histogram-based, fast)
models['hgb'] = HistGradientBoostingClassifier(
    max_iter=300, max_depth=8, learning_rate=0.08,
    min_samples_leaf=30, l2_regularization=1.0,
    random_state=44
)

print(f"\n📋 Models to train: {len(models)}")
for name in models.keys():
    print(f"  - {name.upper()}")

🚀 TRAINING V11 MODELS (10+ Diverse Ensemble)

📋 Models to train: 10
  - XGB1
  - XGB2
  - XGB3
  - LGB1
  - LGB2
  - CAT1
  - CAT2
  - RF
  - ET
  - HGB


In [9]:
# Train all models
print("\n" + "="*70)
print("⏳ Training Models...")
print("="*70)

predictions = {}
probabilities = {}
train_times = {}

model_items = list(models.items())  # Make a copy to avoid modification during iteration

for name, model in model_items:
    print(f"\n  Training {name.upper()}...", end=" ", flush=True)
    start_time = time.time()
    
    try:
        model.fit(X_train_scaled, y_train)
        predictions[name] = model.predict(X_test_scaled)
        probabilities[name] = model.predict_proba(X_test_scaled)
        train_time = time.time() - start_time
        train_times[name] = train_time
        
        test_acc = accuracy_score(y_test, predictions[name])
        print(f"Done! Acc: {test_acc*100:.2f}% (Time: {train_time:.1f}s)")
    except Exception as e:
        print(f"Error: {e}")

print(f"\n✓ Successfully trained {len(predictions)} models")


⏳ Training Models...

  Training XGB1... Done! Acc: 73.75% (Time: 14.3s)

  Training XGB2... Done! Acc: 73.75% (Time: 14.3s)

  Training XGB2... Done! Acc: 73.62% (Time: 16.5s)

  Training XGB3... Done! Acc: 73.62% (Time: 16.5s)

  Training XGB3... Done! Acc: 73.77% (Time: 13.9s)

  Training LGB1... Done! Acc: 73.77% (Time: 13.9s)

  Training LGB1... Done! Acc: 73.72% (Time: 19.9s)

  Training LGB2... Done! Acc: 73.72% (Time: 19.9s)

  Training LGB2... Done! Acc: 73.65% (Time: 22.2s)

  Training CAT1... Done! Acc: 73.65% (Time: 22.2s)

  Training CAT1... Done! Acc: 73.74% (Time: 16.4s)

  Training CAT2... Done! Acc: 73.74% (Time: 16.4s)

  Training CAT2... Done! Acc: 73.69% (Time: 10.8s)

  Training RF... Done! Acc: 73.69% (Time: 10.8s)

  Training RF... Done! Acc: 73.59% (Time: 165.0s)

  Training ET... Done! Acc: 73.59% (Time: 165.0s)

  Training ET... Done! Acc: 73.12% (Time: 36.2s)

  Training HGB... Done! Acc: 73.12% (Time: 36.2s)

  Training HGB... Done! Acc: 73.56% (Time: 51.2s

## 7. Individual Model Performance

In [10]:
print("="*70)
print("📊 INDIVIDUAL MODEL PERFORMANCE")
print("="*70)

results = []
for name in predictions.keys():
    y_pred = predictions[name]
    acc = accuracy_score(y_test, y_pred)
    
    # Per-class metrics
    buy_mask = y_test == 1
    sell_mask = y_test == 2
    hold_mask = y_test == 0
    
    buy_acc = accuracy_score(y_test[buy_mask], y_pred[buy_mask]) if buy_mask.sum() > 0 else 0
    sell_acc = accuracy_score(y_test[sell_mask], y_pred[sell_mask]) if sell_mask.sum() > 0 else 0
    hold_acc = accuracy_score(y_test[hold_mask], y_pred[hold_mask]) if hold_mask.sum() > 0 else 0
    
    results.append({
        'Model': name.upper(),
        'Overall': f"{acc*100:.2f}%",
        'BUY': f"{buy_acc*100:.1f}%",
        'SELL': f"{sell_acc*100:.1f}%",
        'HOLD': f"{hold_acc*100:.1f}%",
        'Time': f"{train_times.get(name, 0):.1f}s"
    })

results_df = pd.DataFrame(results)
print(results_df.to_string(index=False))

📊 INDIVIDUAL MODEL PERFORMANCE
Model Overall   BUY  SELL  HOLD   Time
 XGB1  73.75%  7.4% 13.3% 97.3%  14.3s
 XGB2  73.62% 10.0% 11.9% 96.9%  16.5s
 XGB3  73.77%  6.0% 13.7% 97.6%  13.9s
 LGB1  73.72%  7.6% 13.2% 97.3%  19.9s
 LGB2  73.65%  9.2% 12.4% 97.0%  22.2s
 CAT1  73.74%  8.0% 12.4% 97.4%  16.4s
 CAT2  73.69%  8.7% 12.5% 97.1%  10.8s
   RF  73.59%  3.4% 11.0% 98.3% 165.0s
   ET  73.12%  1.2%  2.7% 99.5%  36.2s
  HGB  73.56%  9.8% 13.0% 96.7%  51.2s


## 8. V11 Ensemble (Weighted Voting)

In [11]:
print("="*70)
print("🔧 BUILDING V11 WEIGHTED ENSEMBLE")
print("="*70)

# Calculate accuracy-based weights
accuracies = {name: accuracy_score(y_test, predictions[name]) for name in predictions.keys()}
total_acc = sum(accuracies.values())
weights = {name: acc / total_acc for name, acc in accuracies.items()}

print("\nModel Weights (accuracy-based):")
for name, w in sorted(weights.items(), key=lambda x: -x[1]):
    print(f"  {name.upper()}: {w:.4f} (acc: {accuracies[name]*100:.2f}%)")

# Weighted Ensemble Probability
final_proba = np.zeros_like(probabilities[list(predictions.keys())[0]])
for name, w in weights.items():
    final_proba += w * probabilities[name]

# Get class probabilities
hold_prob = final_proba[:, 0] * 100  # Class 0 = HOLD
buy_prob = final_proba[:, 1] * 100   # Class 1 = BUY
sell_prob = final_proba[:, 2] * 100  # Class 2 = SELL

# Ensemble predictions
ensemble_pred = np.argmax(final_proba, axis=1)
ensemble_acc = accuracy_score(y_test, ensemble_pred)

print(f"\n✓ Weighted Ensemble Accuracy: {ensemble_acc*100:.2f}%")

🔧 BUILDING V11 WEIGHTED ENSEMBLE

Model Weights (accuracy-based):
  XGB3: 0.1002 (acc: 73.77%)
  XGB1: 0.1002 (acc: 73.75%)
  CAT1: 0.1002 (acc: 73.74%)
  LGB1: 0.1001 (acc: 73.72%)
  CAT2: 0.1001 (acc: 73.69%)
  LGB2: 0.1000 (acc: 73.65%)
  XGB2: 0.1000 (acc: 73.62%)
  RF: 0.1000 (acc: 73.59%)
  HGB: 0.0999 (acc: 73.56%)
  ET: 0.0993 (acc: 73.12%)

✓ Weighted Ensemble Accuracy: 73.72%


## 9. V11 Confidence Calculation

In [14]:
print("="*70)
print("📊 V11 CONFIDENCE ANALYSIS")
print("="*70)

# Debug: Check prediction shapes
print("\nPrediction shapes:")
for name, pred in predictions.items():
    print(f"  {name}: {np.array(pred).shape}")

# Model Agreement Analysis - stack properly
pred_list = []
for name in predictions.keys():
    p = np.array(predictions[name]).flatten()
    pred_list.append(p)

all_preds = np.array(pred_list)
print(f"\nAll predictions shape: {all_preds.shape}")
n_models = len(predictions)

buy_votes = np.sum(all_preds == 1, axis=0)
sell_votes = np.sum(all_preds == 2, axis=0)
hold_votes = np.sum(all_preds == 0, axis=0)

# Agreement bonus
def calculate_confidence_v11(buy_p, sell_p, hold_p, buy_v, sell_v, hold_v, n_models):
    """
    Calculate confidence with agreement bonus
    Returns: (signal_type, confidence)
    """
    max_prob = max(buy_p, sell_p, hold_p)
    
    if buy_p == max_prob:
        signal = 'BUY'
        base_conf = buy_p
        votes = buy_v
    elif sell_p == max_prob:
        signal = 'SELL'
        base_conf = sell_p
        votes = sell_v
    else:
        signal = 'HOLD'
        base_conf = hold_p
        votes = hold_v
    
    # Agreement bonus
    agreement_ratio = votes / n_models
    if agreement_ratio >= 0.9:  # 90%+ models agree
        bonus = 7
    elif agreement_ratio >= 0.7:  # 70%+ models agree
        bonus = 4
    elif agreement_ratio >= 0.5:  # 50%+ models agree
        bonus = 2
    else:
        bonus = 0
    
    confidence = min(100, base_conf + bonus)
    return signal, confidence, votes

# Apply to all test samples
signals = []
confidences = []
vote_counts = []

for i in range(len(y_test)):
    sig, conf, votes = calculate_confidence_v11(
        buy_prob[i], sell_prob[i], hold_prob[i],
        buy_votes[i], sell_votes[i], hold_votes[i],
        n_models
    )
    signals.append(sig)
    confidences.append(conf)
    vote_counts.append(votes)

signals = np.array(signals)
confidences = np.array(confidences)
vote_counts = np.array(vote_counts)

print(f"\nSignal Distribution:")
for sig in ['BUY', 'SELL', 'HOLD']:
    count = (signals == sig).sum()
    print(f"  {sig}: {count:,} ({count/len(signals)*100:.1f}%)")

print(f"\nConfidence Statistics:")
print(f"  Mean: {confidences.mean():.1f}%")
print(f"  Min: {confidences.min():.1f}%")
print(f"  Max: {confidences.max():.1f}%")

📊 V11 CONFIDENCE ANALYSIS

Prediction shapes:
  xgb1: (296579,)
  xgb2: (296579,)
  xgb3: (296579,)
  lgb1: (296579,)
  lgb2: (296579,)
  cat1: (296579, 1)
  cat2: (296579, 1)
  rf: (296579,)
  et: (296579,)
  hgb: (296579,)

All predictions shape: (10, 296579)

Signal Distribution:
  BUY: 6,246 (2.1%)
  SELL: 11,839 (4.0%)
  HOLD: 278,494 (93.9%)

Confidence Statistics:
  Mean: 81.2%
  Min: 33.4%
  Max: 100.0%

Signal Distribution:
  BUY: 6,246 (2.1%)
  SELL: 11,839 (4.0%)
  HOLD: 278,494 (93.9%)

Confidence Statistics:
  Mean: 81.2%
  Min: 33.4%
  Max: 100.0%


## 10. Confidence Threshold Analysis

In [15]:
print("="*70)
print("📈 CONFIDENCE THRESHOLD ANALYSIS")
print("="*70)

# Map signal names to numeric for comparison
signal_map = {'HOLD': 0, 'BUY': 1, 'SELL': 2}
pred_numeric = np.array([signal_map[s] for s in signals])

thresholds = [50, 60, 70, 75, 80, 85, 90, 95]
results = []

for thresh in thresholds:
    mask = confidences >= thresh
    if mask.sum() == 0:
        continue
    
    # Overall
    acc = accuracy_score(y_test[mask], pred_numeric[mask])
    
    # BUY signals only
    buy_mask = mask & (signals == 'BUY')
    buy_correct = (y_test[buy_mask] == 1).sum() if buy_mask.sum() > 0 else 0
    buy_acc = buy_correct / buy_mask.sum() * 100 if buy_mask.sum() > 0 else 0
    
    # SELL signals only
    sell_mask = mask & (signals == 'SELL')
    sell_correct = (y_test[sell_mask] == 2).sum() if sell_mask.sum() > 0 else 0
    sell_acc = sell_correct / sell_mask.sum() * 100 if sell_mask.sum() > 0 else 0
    
    results.append({
        'Threshold': f"{thresh}%+",
        'Signals': mask.sum(),
        'Overall Acc': f"{acc*100:.1f}%",
        'BUY Signals': buy_mask.sum(),
        'BUY Acc': f"{buy_acc:.1f}%",
        'SELL Signals': sell_mask.sum(),
        'SELL Acc': f"{sell_acc:.1f}%"
    })

threshold_df = pd.DataFrame(results)
print(threshold_df.to_string(index=False))

📈 CONFIDENCE THRESHOLD ANALYSIS
Threshold  Signals Overall Acc  BUY Signals BUY Acc  SELL Signals SELL Acc
     50%+   268046       77.7%          575   44.0%          3027    40.7%
     60%+   242593       81.1%           25   24.0%           424    43.4%
     70%+   213525       84.5%            2    0.0%            37    62.2%
     75%+   196135       86.2%            0    0.0%             9    66.7%
     80%+   177722       87.9%            0    0.0%             0     0.0%
     85%+   157840       89.6%            0    0.0%             0     0.0%
     90%+   136722       91.3%            0    0.0%             0     0.0%
     95%+   110476       92.8%            0    0.0%             0     0.0%


## 10.5 Alternative: Binary BUY/SELL Models (Skip HOLD)

Multi-class дээр BUY/SELL accuracy маш бага байна. Өөр арга - BUY болон SELL-ийг binary classification хэлбэрээр сургах.

In [16]:
print("="*70)
print("🔄 ALTERNATIVE: BINARY BUY vs SELL MODEL")
print("="*70)

# Filter only BUY (1) and SELL (2), skip HOLD (0)
train_binary = train_df[train_df['signal'] != 0].copy()
test_binary = test_df[test_df['signal'] != 0].copy()

# Convert to binary: BUY=1, SELL=0
train_binary['signal_binary'] = (train_binary['signal'] == 1).astype(int)
test_binary['signal_binary'] = (test_binary['signal'] == 1).astype(int)

print(f"Train (BUY/SELL only): {len(train_binary):,}")
print(f"Test (BUY/SELL only): {len(test_binary):,}")
print(f"Train BUY ratio: {train_binary['signal_binary'].mean()*100:.1f}%")
print(f"Test BUY ratio: {test_binary['signal_binary'].mean()*100:.1f}%")

# Prepare data
X_train_bin = train_binary[selected_features].values
y_train_bin = train_binary['signal_binary'].values
X_test_bin = test_binary[selected_features].values
y_test_bin = test_binary['signal_binary'].values

# Scale
X_train_bin_scaled = scaler.transform(X_train_bin)
X_test_bin_scaled = scaler.transform(X_test_bin)

🔄 ALTERNATIVE: BINARY BUY vs SELL MODEL
Train (BUY/SELL only): 393,249
Test (BUY/SELL only): 80,296
Train BUY ratio: 49.3%
Test BUY ratio: 52.2%
Train (BUY/SELL only): 393,249
Test (BUY/SELL only): 80,296
Train BUY ratio: 49.3%
Test BUY ratio: 52.2%


In [17]:
# Train binary models (BUY vs SELL)
print("\n⏳ Training Binary BUY/SELL Models...")

binary_models = {}

# XGBoost models
xgb_bin_params = {'verbosity': 0}
if GPU_AVAILABLE:
    xgb_bin_params.update({'tree_method': 'hist', 'device': 'cuda'})

binary_models['xgb1'] = xgb.XGBClassifier(
    n_estimators=400, max_depth=6, learning_rate=0.04,
    subsample=0.8, colsample_bytree=0.8,
    reg_alpha=0.1, reg_lambda=1.0,
    random_state=42, **xgb_bin_params
)

binary_models['xgb2'] = xgb.XGBClassifier(
    n_estimators=300, max_depth=8, learning_rate=0.05,
    subsample=0.75, colsample_bytree=0.75,
    random_state=43, **xgb_bin_params
)

binary_models['xgb3'] = xgb.XGBClassifier(
    n_estimators=500, max_depth=4, learning_rate=0.03,
    subsample=0.85, colsample_bytree=0.85,
    random_state=44, **xgb_bin_params
)

# LightGBM models
lgb_bin_params = {'verbose': -1}
if GPU_AVAILABLE:
    lgb_bin_params['device'] = 'gpu'

binary_models['lgb1'] = lgb.LGBMClassifier(
    n_estimators=400, max_depth=6, learning_rate=0.04,
    subsample=0.8, colsample_bytree=0.8,
    num_leaves=31, random_state=42, **lgb_bin_params
)

binary_models['lgb2'] = lgb.LGBMClassifier(
    n_estimators=300, max_depth=8, learning_rate=0.05,
    num_leaves=63, random_state=45, **lgb_bin_params
)

# CatBoost models
cat_bin_params = {'verbose': False}
if GPU_AVAILABLE:
    cat_bin_params.update({'task_type': 'GPU', 'devices': '0'})

binary_models['cat1'] = CatBoostClassifier(
    iterations=400, depth=6, learning_rate=0.04,
    random_seed=42, **cat_bin_params
)

binary_models['cat2'] = CatBoostClassifier(
    iterations=300, depth=8, learning_rate=0.05,
    random_seed=46, **cat_bin_params
)

# Train all binary models
bin_predictions = {}
bin_probabilities = {}

for name, model in binary_models.items():
    print(f"  Training {name.upper()}...", end=" ", flush=True)
    start = time.time()
    model.fit(X_train_bin_scaled, y_train_bin)
    bin_predictions[name] = model.predict(X_test_bin_scaled)
    bin_probabilities[name] = model.predict_proba(X_test_bin_scaled)
    acc = accuracy_score(y_test_bin, bin_predictions[name])
    print(f"Done! Acc: {acc*100:.2f}% (Time: {time.time()-start:.1f}s)")

print(f"\n✓ Binary models trained: {len(binary_models)}")


⏳ Training Binary BUY/SELL Models...
  Training XGB1... Done! Acc: 51.33% (Time: 2.8s)
  Training XGB2... Done! Acc: 51.33% (Time: 2.8s)
  Training XGB2... Done! Acc: 50.66% (Time: 3.3s)
  Training XGB3... Done! Acc: 50.66% (Time: 3.3s)
  Training XGB3... Done! Acc: 51.75% (Time: 2.4s)
  Training LGB1... Done! Acc: 51.75% (Time: 2.4s)
  Training LGB1... Done! Acc: 51.47% (Time: 3.9s)
  Training LGB2... Done! Acc: 51.47% (Time: 3.9s)
  Training LGB2... Done! Acc: 51.08% (Time: 4.9s)
  Training CAT1... Done! Acc: 51.08% (Time: 4.9s)
  Training CAT1... Done! Acc: 50.84% (Time: 4.0s)
  Training CAT2... Done! Acc: 50.84% (Time: 4.0s)
  Training CAT2... Done! Acc: 52.44% (Time: 4.0s)

✓ Binary models trained: 7
Done! Acc: 52.44% (Time: 4.0s)

✓ Binary models trained: 7


In [18]:
# Binary Ensemble
print("="*70)
print("📊 BINARY ENSEMBLE RESULTS (BUY vs SELL)")
print("="*70)

# Calculate weights
bin_accuracies = {name: accuracy_score(y_test_bin, bin_predictions[name]) for name in binary_models.keys()}
total_acc = sum(bin_accuracies.values())
bin_weights = {name: acc / total_acc for name, acc in bin_accuracies.items()}

print("\nModel Weights:")
for name, w in sorted(bin_weights.items(), key=lambda x: -x[1]):
    print(f"  {name.upper()}: {w:.4f} (acc: {bin_accuracies[name]*100:.2f}%)")

# Weighted probability
bin_final_proba = np.zeros_like(bin_probabilities['xgb1'])
for name, w in bin_weights.items():
    bin_final_proba += w * bin_probabilities[name]

bin_buy_prob = bin_final_proba[:, 1] * 100  # BUY probability
bin_sell_prob = bin_final_proba[:, 0] * 100  # SELL probability

# Model agreement
bin_all_preds = np.array([bin_predictions[name].flatten() for name in binary_models.keys()])
bin_buy_votes = np.sum(bin_all_preds == 1, axis=0)
n_bin_models = len(binary_models)

# Calculate confidence with agreement bonus
def calc_binary_confidence(buy_p, buy_v, n_models):
    """BUY probability with agreement bonus"""
    if buy_p >= 50:
        signal = 'BUY'
        base_conf = buy_p
        votes = buy_v
    else:
        signal = 'SELL'
        base_conf = 100 - buy_p
        votes = n_models - buy_v
    
    agreement = votes / n_models
    if agreement >= 0.85:
        bonus = 7
    elif agreement >= 0.7:
        bonus = 4
    elif agreement >= 0.5:
        bonus = 2
    else:
        bonus = 0
    
    return signal, min(100, base_conf + bonus), votes

# Apply
bin_signals = []
bin_confidences = []
bin_votes = []

for i in range(len(y_test_bin)):
    sig, conf, votes = calc_binary_confidence(bin_buy_prob[i], bin_buy_votes[i], n_bin_models)
    bin_signals.append(sig)
    bin_confidences.append(conf)
    bin_votes.append(votes)

bin_signals = np.array(bin_signals)
bin_confidences = np.array(bin_confidences)

print(f"\nBinary Signal Distribution:")
print(f"  BUY: {(bin_signals == 'BUY').sum():,} ({(bin_signals == 'BUY').mean()*100:.1f}%)")
print(f"  SELL: {(bin_signals == 'SELL').sum():,} ({(bin_signals == 'SELL').mean()*100:.1f}%)")
print(f"\nConfidence: Mean={bin_confidences.mean():.1f}%, Min={bin_confidences.min():.1f}%, Max={bin_confidences.max():.1f}%")

📊 BINARY ENSEMBLE RESULTS (BUY vs SELL)

Model Weights:
  CAT2: 0.1458 (acc: 52.44%)
  XGB3: 0.1439 (acc: 51.75%)
  LGB1: 0.1431 (acc: 51.47%)
  XGB1: 0.1427 (acc: 51.33%)
  LGB2: 0.1421 (acc: 51.08%)
  CAT1: 0.1414 (acc: 50.84%)
  XGB2: 0.1409 (acc: 50.66%)

Binary Signal Distribution:
  BUY: 39,016 (48.6%)
  SELL: 41,280 (51.4%)

Confidence: Mean=60.8%, Min=50.0%, Max=99.0%


In [19]:
# Binary Confidence Threshold Analysis
print("="*70)
print("📈 BINARY CONFIDENCE THRESHOLD ANALYSIS")
print("="*70)

# y_test_bin: 1=BUY, 0=SELL
# bin_signals: 'BUY' or 'SELL'
bin_pred_numeric = (bin_signals == 'BUY').astype(int)

thresholds = [50, 60, 70, 75, 80, 85, 90, 95]
bin_results = []

for thresh in thresholds:
    mask = bin_confidences >= thresh
    if mask.sum() == 0:
        continue
    
    # Overall accuracy
    acc = accuracy_score(y_test_bin[mask], bin_pred_numeric[mask])
    
    # BUY accuracy
    buy_mask = mask & (bin_signals == 'BUY')
    if buy_mask.sum() > 0:
        buy_correct = (y_test_bin[buy_mask] == 1).sum()
        buy_acc = buy_correct / buy_mask.sum() * 100
    else:
        buy_acc = 0
    
    # SELL accuracy
    sell_mask = mask & (bin_signals == 'SELL')
    if sell_mask.sum() > 0:
        sell_correct = (y_test_bin[sell_mask] == 0).sum()
        sell_acc = sell_correct / sell_mask.sum() * 100
    else:
        sell_acc = 0
    
    bin_results.append({
        'Threshold': f"{thresh}%+",
        'Signals': mask.sum(),
        'Overall': f"{acc*100:.1f}%",
        'BUY': buy_mask.sum(),
        'BUY Acc': f"{buy_acc:.1f}%",
        'SELL': sell_mask.sum(),
        'SELL Acc': f"{sell_acc:.1f}%"
    })

bin_thresh_df = pd.DataFrame(bin_results)
print(bin_thresh_df.to_string(index=False))

📈 BINARY CONFIDENCE THRESHOLD ANALYSIS
Threshold  Signals Overall   BUY BUY Acc  SELL SELL Acc
     50%+    80296   51.5% 39016   53.8% 41280    49.4%
     60%+    45254   52.0% 22346   54.4% 22908    49.7%
     70%+     5046   53.3%  2386   57.3%  2660    49.7%
     75%+     1612   54.0%   692   57.7%   920    51.2%
     80%+      602   57.1%   263   63.9%   339    51.9%
     85%+      204   59.8%    65   84.6%   139    48.2%
     90%+       71   85.9%    20  100.0%    51    80.4%
     95%+       11  100.0%     0    0.0%    11   100.0%


In [20]:
# Binary Backtest
def backtest_binary_v11(test_df_bin, signals, confidences, y_test, threshold=75, sl_atr_mult=1.5, tp_atr_mult=2.5):
    """
    Backtest for binary BUY/SELL predictions
    """
    results = {
        'total': 0, 'wins': 0, 'losses': 0,
        'buy_signals': 0, 'buy_wins': 0,
        'sell_signals': 0, 'sell_wins': 0,
        'total_pips': 0, 'buy_pips': 0, 'sell_pips': 0
    }
    
    test_clean = test_df_bin.reset_index(drop=True)
    atr_values = test_clean['atr'].values if 'atr' in test_clean.columns else np.full(len(test_clean), 0.0008)
    
    for i in range(len(signals)):
        if confidences[i] < threshold:
            continue
        
        results['total'] += 1
        actual_buy = (y_test[i] == 1)
        predicted_buy = (signals[i] == 'BUY')
        
        atr = atr_values[i] if i < len(atr_values) else 0.0008
        tp_pips = atr * tp_atr_mult * 10000
        sl_pips = atr * sl_atr_mult * 10000
        
        if predicted_buy:
            results['buy_signals'] += 1
            if actual_buy:
                results['wins'] += 1
                results['buy_wins'] += 1
                results['total_pips'] += tp_pips
                results['buy_pips'] += tp_pips
            else:
                results['losses'] += 1
                results['total_pips'] -= sl_pips
                results['buy_pips'] -= sl_pips
        else:
            results['sell_signals'] += 1
            if not actual_buy:  # actual is SELL
                results['wins'] += 1
                results['sell_wins'] += 1
                results['total_pips'] += tp_pips
                results['sell_pips'] += tp_pips
            else:
                results['losses'] += 1
                results['total_pips'] -= sl_pips
                results['sell_pips'] -= sl_pips
    
    results['win_rate'] = results['wins'] / results['total'] * 100 if results['total'] > 0 else 0
    results['buy_wr'] = results['buy_wins'] / results['buy_signals'] * 100 if results['buy_signals'] > 0 else 0
    results['sell_wr'] = results['sell_wins'] / results['sell_signals'] * 100 if results['sell_signals'] > 0 else 0
    
    return results

# Run backtest
print("="*70)
print("📈 BINARY BACKTEST RESULTS")
print("="*70)

bt_results = []
for thresh in [60, 70, 75, 80, 85, 90, 95]:
    res = backtest_binary_v11(test_binary, bin_signals, bin_confidences, y_test_bin, threshold=thresh)
    if res['total'] > 0:
        pf = (res['wins'] * 20) / (res['losses'] * 12 + 1) if res['losses'] > 0 else float('inf')
        bt_results.append({
            'Thresh': f"{thresh}%+",
            'Signals': res['total'],
            'BUY': res['buy_signals'],
            'SELL': res['sell_signals'],
            'Win Rate': f"{res['win_rate']:.1f}%",
            'BUY WR': f"{res['buy_wr']:.1f}%",
            'SELL WR': f"{res['sell_wr']:.1f}%",
            'Pips': f"{res['total_pips']:+,.0f}",
            'PF': f"{pf:.1f}" if pf != float('inf') else "∞"
        })

bt_df = pd.DataFrame(bt_results)
print(bt_df.to_string(index=False))

📈 BINARY BACKTEST RESULTS
Thresh  Signals   BUY  SELL Win Rate BUY WR SELL WR    Pips   PF
  60%+    45254 22346 22908    52.0%  54.4%   49.7% +73,438  1.8
  70%+     5046  2386  2660    53.3%  57.3%   49.7% +12,527  1.9
  75%+     1612   692   920    54.0%  57.7%   51.2%  +5,802  2.0
  80%+      602   263   339    57.1%  63.9%   51.9%  +3,178  2.2
  85%+      204    65   139    59.8%  84.6%   48.2%  +1,614  2.5
  90%+       71    20    51    85.9% 100.0%   80.4%  +1,369 10.1
  95%+       11     0    11   100.0%   0.0%  100.0%    +445    ∞


In [21]:
# Save V11 Binary Models
print("="*70)
print("💾 SAVING V11 BINARY MODELS")
print("="*70)

# Save binary models
for name, model in binary_models.items():
    model_path = MODEL_DIR / f"{name}_v11_binary.joblib"
    joblib.dump(model, model_path)
    print(f"  ✓ Saved {name}_v11_binary.joblib")

# Save scaler
joblib.dump(scaler, MODEL_DIR / 'scaler_v11.joblib')
print(f"  ✓ Saved scaler_v11.joblib")

# Save feature columns
joblib.dump(selected_features, MODEL_DIR / 'feature_cols_v11.joblib')
print(f"  ✓ Saved feature_cols_v11.joblib")

# Save weights
joblib.dump(bin_weights, MODEL_DIR / 'weights_v11.joblib')
print(f"  ✓ Saved weights_v11.joblib")

# Save config
config = {
    'version': 'v11',
    'model_type': 'binary',  # BUY vs SELL
    'n_models': len(binary_models),
    'model_names': list(binary_models.keys()),
    'n_classes': 2,
    'class_mapping': {0: 'SELL', 1: 'BUY'},
    'n_features': len(selected_features),
    'weights': bin_weights,
    'gpu_available': GPU_AVAILABLE,
    'labeling_params': {
        'forward_periods': 60,
        'min_pips': 15,
        'ratio': 1.5
    },
    'performance': {
        '90%+': {'signals': 71, 'accuracy': 85.9, 'pips': 1369, 'pf': 10.1},
        '85%+': {'signals': 204, 'accuracy': 59.8, 'pips': 1614, 'pf': 2.5},
        '80%+': {'signals': 602, 'accuracy': 57.1, 'pips': 3178, 'pf': 2.2}
    }
}
joblib.dump(config, MODEL_DIR / 'config_v11.joblib')
print(f"  ✓ Saved config_v11.joblib")

print(f"\n✅ All V11 models saved to {MODEL_DIR}")

💾 SAVING V11 BINARY MODELS
  ✓ Saved xgb1_v11_binary.joblib
  ✓ Saved xgb2_v11_binary.joblib
  ✓ Saved xgb3_v11_binary.joblib
  ✓ Saved lgb1_v11_binary.joblib
  ✓ Saved lgb2_v11_binary.joblib
  ✓ Saved cat1_v11_binary.joblib
  ✓ Saved cat2_v11_binary.joblib
  ✓ Saved scaler_v11.joblib
  ✓ Saved feature_cols_v11.joblib
  ✓ Saved weights_v11.joblib
  ✓ Saved config_v11.joblib

✅ All V11 models saved to c:\Users\Acer\Desktop\Forex-Signal-App\models\signal_generator_v11


## 11. Backtest Simulation

In [22]:
def backtest_v11(test_df, signals, confidences, y_test, threshold=75, sl_atr_mult=1.5, tp_atr_mult=2.5):
    """
    V11 Backtest with BUY and SELL signals
    """
    results = {
        'total_signals': 0,
        'buy_signals': 0,
        'sell_signals': 0,
        'wins': 0,
        'losses': 0,
        'buy_wins': 0,
        'buy_losses': 0,
        'sell_wins': 0,
        'sell_losses': 0,
        'total_pips': 0,
        'buy_pips': 0,
        'sell_pips': 0
    }
    
    test_clean = test_df.reset_index(drop=True)
    atr_values = test_clean['atr'].values if 'atr' in test_clean.columns else np.full(len(test_clean), 0.0008)
    
    for i in range(len(signals)):
        if confidences[i] < threshold:
            continue
        if signals[i] == 'HOLD':
            continue
        
        results['total_signals'] += 1
        
        # Determine if prediction was correct
        actual = y_test[i]
        
        if signals[i] == 'BUY':
            results['buy_signals'] += 1
            is_correct = (actual == 1)  # BUY was correct
            
            if is_correct:
                results['wins'] += 1
                results['buy_wins'] += 1
                pips = atr_values[i] * tp_atr_mult * 10000
                results['total_pips'] += pips
                results['buy_pips'] += pips
            else:
                results['losses'] += 1
                results['buy_losses'] += 1
                pips = atr_values[i] * sl_atr_mult * 10000
                results['total_pips'] -= pips
                results['buy_pips'] -= pips
        
        elif signals[i] == 'SELL':
            results['sell_signals'] += 1
            is_correct = (actual == 2)  # SELL was correct
            
            if is_correct:
                results['wins'] += 1
                results['sell_wins'] += 1
                pips = atr_values[i] * tp_atr_mult * 10000
                results['total_pips'] += pips
                results['sell_pips'] += pips
            else:
                results['losses'] += 1
                results['sell_losses'] += 1
                pips = atr_values[i] * sl_atr_mult * 10000
                results['total_pips'] -= pips
                results['sell_pips'] -= pips
    
    # Calculate win rates
    results['win_rate'] = results['wins'] / results['total_signals'] * 100 if results['total_signals'] > 0 else 0
    results['buy_win_rate'] = results['buy_wins'] / results['buy_signals'] * 100 if results['buy_signals'] > 0 else 0
    results['sell_win_rate'] = results['sell_wins'] / results['sell_signals'] * 100 if results['sell_signals'] > 0 else 0
    
    return results

# Run backtest for different thresholds
print("="*70)
print("📈 BACKTEST RESULTS")
print("="*70)

backtest_results = []
for thresh in [70, 75, 80, 85, 90]:
    res = backtest_v11(test_df, signals, confidences, y_test, threshold=thresh)
    if res['total_signals'] > 0:
        backtest_results.append({
            'Threshold': f"{thresh}%+",
            'Signals': res['total_signals'],
            'BUY': res['buy_signals'],
            'SELL': res['sell_signals'],
            'Win Rate': f"{res['win_rate']:.1f}%",
            'BUY WR': f"{res['buy_win_rate']:.1f}%",
            'SELL WR': f"{res['sell_win_rate']:.1f}%",
            'Pips': f"{res['total_pips']:+.1f}"
        })

bt_df = pd.DataFrame(backtest_results)
print(bt_df.to_string(index=False))

📈 BACKTEST RESULTS
Threshold  Signals  BUY  SELL Win Rate BUY WR SELL WR   Pips
     70%+       39    2    37    59.0%   0.0%   62.2% +662.2
     75%+        9    0     9    66.7%   0.0%   66.7% +120.5
Threshold  Signals  BUY  SELL Win Rate BUY WR SELL WR   Pips
     70%+       39    2    37    59.0%   0.0%   62.2% +662.2
     75%+        9    0     9    66.7%   0.0%   66.7% +120.5


## 12. Save Models

In [ ]:
print("="*70)
print("💾 SAVING V11 MODELS")
print("="*70)

# Save all trained models
for name, model in models.items():
    model_path = MODEL_DIR / f"{name}_v11.joblib"
    joblib.dump(model, model_path)
    print(f"  ✓ Saved {name}_v11.joblib")

# Save scaler
joblib.dump(scaler, MODEL_DIR / 'scaler_v11.joblib')
print(f"  ✓ Saved scaler_v11.joblib")

# Save feature columns
joblib.dump(selected_features, MODEL_DIR / 'feature_cols_v11.joblib')
print(f"  ✓ Saved feature_cols_v11.joblib")

# Save weights
joblib.dump(weights, MODEL_DIR / 'weights_v11.joblib')
print(f"  ✓ Saved weights_v11.joblib")

# Save config
config = {
    'version': 'v11',
    'n_models': len(models),
    'model_names': list(models.keys()),
    'n_classes': 3,
    'class_mapping': {0: 'HOLD', 1: 'BUY', 2: 'SELL'},
    'n_features': len(selected_features),
    'weights': weights,
    'gpu_available': GPU_AVAILABLE,
    'labeling_params': {
        'forward_periods': 60,
        'min_pips': 15,
        'ratio': 1.5
    }
}
joblib.dump(config, MODEL_DIR / 'config_v11.joblib')
print(f"  ✓ Saved config_v11.joblib")

print(f"\n✓ All models saved to {MODEL_DIR}")

## 13. Summary

In [ ]:
print("\n" + "="*70)
print("📊 V11 TRAINING SUMMARY")
print("="*70)

print(f"""
🎯 V11 Multi-Class Signal Generator

Models Trained: {len(models)}
  - XGBoost: 3 variants
  - LightGBM: 2 variants
  - CatBoost: 2 variants
  - RandomForest: 1
  - ExtraTrees: 1
  - HistGradientBoosting: 1

Features: {len(selected_features)}
Classes: 3 (HOLD, BUY, SELL)

Ensemble Accuracy: {ensemble_acc*100:.2f}%

Best Individual Model:
  {max(accuracies, key=accuracies.get).upper()}: {max(accuracies.values())*100:.2f}%

GPU Used: {GPU_AVAILABLE}
""")

print("\n✅ V11 Training Complete!")